<a href="https://colab.research.google.com/github/drbarak/NLP-FinalProject/blob/main/ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
INSTALL = False
if INSTALL:
  # to switch version of spacy
  !pip uninstall spacy
  !pip uninstall en_core_web_mdy
  
  !pip install spacy==2.2.4 #3.0.6

  !pip install pyforest
  #!pip install wget
  !python -m spacy download en_core_web_md

# INIT

In [3]:
from importlib import import_module
p = print
def my_import(lib, package=None, func=None, version=None):
  try:
    import_module(lib)
  except:
    if package is None: package = lib
    if version:
      p('pip install "{package}=={version}"')
      !pip install "{package}=={version}"
    else:
      p('pip install "{package}"')
      !pip install "{package}"
    import_module(lib)
  
  if func is not None:
    return import_module(lib).__getattribute__(func)

In [4]:
my_import('pyforest')
import json
import string
import random
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

p = print
d = display

pip install "{package}"
  Created wheel for pyforest: filename=pyforest-1.1.0-py2.py3-none-any.whl size=14606 sha256=926ba467a4bfe25806726db2a1c5746d1ffd251336f804aaac8fc1de855cad5e
  Stored in directory: /root/.cache/pip/wheels/40/c6/da/43562aeea85b37f1a2b3d326f0f602f865000d2ada8a43625f
Successfully built pyforest


In [5]:
#my_import('spacy', version='3.0.6')
p('spacy.__version__', spacy.__version__)
from spacy.tokens import Doc, Span
from spacy.matcher import PhraseMatcher
from spacy.language import Language

# Load md (medium) English tokenizer, tagger, parser and NER, and vectors(vectors are not included in the 'sm' model)
#nlp = spacy.load('en_core_web_md')  NOT WORKING ON COLAB
try:
  import en_core_web_md
except:  
  !python -m spacy download en_core_web_md
  import en_core_web_md
  
nlp = en_core_web_md.load()

<IPython.core.display.Javascript object>

spacy.__version__ 2.2.4
     |████████████████████████████████| 96.4MB 1.3MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.5-cp37-none-any.whl size=98051304 sha256=2398c1e5bd38920b8130d4e6ce46b1d00c0b8e76f38fffa5055d3588f64de7a1
  Stored in directory: /tmp/pip-ephem-wheel-cache-cnrhhijc/wheels/df/94/ad/f5cf59224cea6b5686ac4fd1ad19c8a07bc026e13c36502d81
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [6]:
#!pip uninstall googletrans
my_import('googletrans', version='3.1.0a0')
from googletrans import Translator
translator = Translator()

pip install "{package}=={version}"
     |████████████████████████████████| 61kB 3.9MB/s 
     |████████████████████████████████| 51kB 5.7MB/s 
     |████████████████████████████████| 1.0MB 12.8MB/s 
     |████████████████████████████████| 61kB 7.3MB/s 
     |████████████████████████████████| 71kB 6.9MB/s 
  Created wheel for googletrans: filename=googletrans-3.1.0a0-cp37-none-any.whl size=16368 sha256=f9d290796399519658ac9b0f9ff2a3af74448b7dc38a53205194abae4e6203c9
  Stored in directory: /root/.cache/pip/wheels/27/7a/a0/aff3babbb775549ce6813cb8fa7ff3c0848c4dc62c20f8fdac
Successfully built googletrans


In [7]:
#translation = translator.translate("Hola Mundo", dest="he") # default is english, can specify source langauge ,src="de"
#print(f"{translation.origin} ({translation.src}) --> {translation.text} ({translation.dest})")

In [8]:
import requests

api_key = 'c2adfa29edfd95ad16efab9218619ff3'
URL = "http://api.openweathermap.org/data/2.5/weather?"

In [9]:
path = '/content/drive/MyDrive/Project5_ChatBot/'
data_url = 'https://raw.githubusercontent.com/DanielKorenDataScience/NLP-FinalProject/main/data/' 

In [10]:
TEST = False

In [11]:
def process_text(text):
    doc = nlp(text.lower())
    result = []
    for token in doc:
        if token.text in nlp.Defaults.stop_words:   # len(nlp.Defaults.stop_words)=326
            continue
        if token.is_punct:
            continue
        if token.lemma_ == '-PRON-':
            p('-PRON-: ', token.lemma_)
            continue
        result.append(token.lemma_)
    return " ".join(result)

In [12]:
def calculate_similarity(text1, text2):
    def get_similarity(doc1, doc2):
      if doc1 and doc1.vector_norm:
        if doc2 and doc2.vector_norm:
          return doc1.similarity(doc2)
      return 0

    doc1 = nlp(text1)
    #p(doc1.pos_)
    doc2 = nlp(text2)
    similarity1 = get_similarity(doc1, doc2)

    doc1 = nlp(process_text(text1))
    #p(doc1)
    doc2 = nlp(process_text(text2))
    #p(doc2)
    similarity2 = get_similarity(doc1, doc2)

    return max(similarity1, similarity2)

# Call the web site

{"coord":{"lon":34.8,"lat":32.0833},

"weather":[{"id":801,"main":"Clouds","description":"few clouds","icon":"02n"}],

"base":"stations",

"main":{"temp":295.69,"feels_like":295.83,"temp_min":294.42,"temp_max":296.58,"pressure":1016,"humidity":70},

"visibility":10000,

"wind":{"speed":2.06,"deg":10},

"clouds":{"all":20},

"dt":1623961727,

"sys":{"type":1,"id":6845,"country":"IL","sunrise":1623897263,"sunset":1623948546},

"timezone":10800,

"id":293396,

"name":"Tel Aviv",

"cod":200}

In [14]:
WEATHER_ONLY = 1

In [15]:
def call_web(city_name, country, count=1):
    state = ''
    api_url = f"{URL}q={city_name},{state},{country}&cnt={count}&appid={api_key}"
    #p(api_url)

    response = requests.get(api_url)
    response_dict = response.json()

    weather = response_dict #["weather"]#[0]#["description"]

    if response.status_code == 200:
        return weather
    elif response.status_code == 404:
        return None
    else:
        print('[!] HTTP {0} calling [{1}]'.format(response.status_code, api_url))
        return None

In [116]:
def get_weather(dt, location, action = None):
  template = WEATHER_ONLY
  #p(dt, location)
  city_w = city = location[0][0]  # assume one city (not handling if there are 2 cities)
  #p(city, city_w)
  result = ''
  # check if city in api_city list
  if city in df_CITIES_API.index.values:
    result = df_CITIES_API.loc[city]
  if len(result) == 0: # not found, get district
    if city in cities_il_df.index.values:
      city_w = cities_il_df.loc[city].district
  elif len(result) > 0: # more than one city -> need country
    pass

  #p(city_w, location)
  country = location[0][1]
  #p(country, len(countries_df.Name))
  if country in list(countries_df.Name):
    country_code = countries_df[countries_df.Name == country].index[0]
  else:
    country_code = ''
  #p(city_w, country_code)#
  city_weather = call_web(city_w, country_code)
  if city_weather is not None:
    if template == WEATHER_ONLY:
      result = city_weather["weather"][0]["description"]
    return f"In {city.capitalize()}, {location[0][1].capitalize()} the current weather is: {result}"
  else:
    return "Something went wrong (API)."
 
#answer = get_weather('current', [('toronto', 'united states')])
#answer

# Get cities and update model

In [18]:
GIT = True
git_path = 'https://raw.githubusercontent.com/DanielKorenDataScience/NLP-FinalProject/main/data/'

def get_json(fname, GIT=True):
  p(fname)
  if GIT:
    resp = requests.get(data_url + fname)
    return json.loads(resp.text)
  with open(path + fname, encoding="utf8") as f:
    return json.loads(f.read())

In [19]:
intents = get_json('intents.json')
#intents

intents.json


In [20]:
my_import('wget')
import wget

def load_CITIES_API():
  # file too big for git so read it from my google drive using a shared link
  CITIES_API = get_json('city.list.json')
  ''' 
  git_path = 'https://raw.githubusercontent.com/DanielKorenDataScience/NLP-FinalProject/main/' + 'city.list.json'
  resp = requests.get(git_path)
  CITIES_API = json.loads(resp.text)

  google_shared_link = 'https://drive.google.com/file/d/19lB4T32o1VbFAKRk_PExbNYV8mcZ8hRT/view?usp=sharing'
  url = 'https://drive.google.com/uc?authuser=0&id=19lB4T32o1VbFAKRk_PExbNYV8mcZ8hRT&export=download'
  file_id = '19lB4T32o1VbFAKRk_PExbNYV8mcZ8hRT'
  
  while True:
    try:
      with open('city.list.json', encoding="utf8") as f:
        CITIES_API = json.loads(f.read())
      break
    except:
      wget.download(url)  # download the file to local google drive with the original name in the share link
      # now can open with no path because it is in the local 'colab' directory
  
  '''
  df = pd.DataFrame(CITIES_API)
  df = df.drop(['id', 'coord'], axis='columns')
  df.name = df.name.str.lower()
  df.name = df.name.str.translate(remove_punct_dict)

  CITIES_API_city = sorted(set(df.name.to_list()))
  CITIES_API_country = sorted(set(df.country.str.lower().to_list()))
  if CITIES_API_country[0] == '':
    CITIES_API_country = CITIES_API_country[1:]
  CITIES_API_state = sorted(set(df.state.str.lower().to_list()))
  if CITIES_API_state[0] == '':
    CITIES_API_state = CITIES_API_state[1:]
  if CITIES_API_state[0] == '00':
    CITIES_API_state = CITIES_API_state[1:]
  
  df.set_index('name', inplace=True)
  #d(df.head)
  return df, CITIES_API_city, CITIES_API_country, CITIES_API_state

df_CITIES_API, CITIES_API_city, CITIES_API_country_code, CITIES_API_state_code = load_CITIES_API()
df_CITIES_API.head()

pip install "{package}"
  Created wheel for wget: filename=wget-3.2-cp37-none-any.whl size=9675 sha256=a50d583c13e9de0d23fccb8492e5feac3baa67a049dbfc275a3d23b72e6ac60b
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget
city.list.json


<IPython.core.display.Javascript object>

,state,country
name,,
ḩeşāre sefīd,,IR
‘ayn ḩalāqīm,,SY
taglag,,IR
qabāghlū,,IR
‘arīqah,,SY


In [21]:
def load_countries():
  path_ = path  
  fname = 'countries.csv'
  if GIT:
    path_ = git_path
  df = pd.read_csv(path_ + fname)
  df.Name = df.Name.str.lower()
  COUNTRIES = df.Name.tolist()
  df = df.set_index('Code')
  return df, COUNTRIES

countries_df, COUNTRIES = load_countries()
countries_df.head()

<IPython.core.display.Javascript object>

,Name
Code,
AF,afghanistan
AX,åland islands
AL,albania
DZ,algeria
AS,american samoa


In [22]:
countries_df[countries_df.Name == 'israel'].iloc[0].name
countries_df.loc['US']

Name    united states
Name: US, dtype: object

In [23]:
def load_cities_il():
  path_ = path
  fname = 'cities_IL.csv'
  if GIT:
    path_ = git_path
  df = pd.read_csv(path_ + fname, names=['city', 'district'])
  df.city = df.city.str.translate(remove_punct_dict)
  CITIES = df.city.tolist()
  df = df.set_index('city')
  return df, CITIES
cities_il_df, CITIES_IL = load_cities_il()
cities_il_df.head()

<IPython.core.display.Javascript object>

,district
city,
asam,beersheba
abbirim,acre
abu abdun,beersheba
abu ammar,beersheba
abu amre,beersheba


In [24]:
def load_capitals():
  path_ = path
  fname = 'capitals.json'
  if GIT:
    path_ = git_path
  CAPITALS = pd.read_json(path_ + fname, encoding="utf8", typ='series').str.lower()

  CAPITALS.index = CAPITALS.index.str.lower()
  df = pd.DataFrame([CAPITALS])
  df = df.T
  df = df.reset_index()
  df.columns = ['country', 'capital']
  df = df.set_index('capital')
  return df, CAPITALS.to_dict()
capitals_df, CAPITALS = load_capitals()

p(CAPITALS['albania'])
p(capitals_df.loc['madrid'].country)
capitals_df.head()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

tirana
spain


,country
capital,
kabul,afghanistan
mariehamn,åland islands
tirana,albania
algiers,algeria
pago pago,american samoa


In [25]:
def load_largest():
  fname = '100-largest-cities.csv'
  path_ = path
  if GIT:
    path_ = git_path
  df = pd.read_csv(path_ + fname, encoding='ISO-8859-8')
  df.city = df.city.str.lower()
  df.city = df.city.str.translate(remove_punct_dict)
  LARGEST = df.city.tolist()
  df = df.set_index('city')
  return df, LARGEST
largest_df, LARGEST = load_largest()

d(largest_df.head())
p(largest_df.loc['toronto'].country)

<IPython.core.display.Javascript object>

,country
city,
tokyo,Japan
delhi,India
shanghai,China
so paulo,Brazil
ciudad de mxico mexico city,Mexico


Canada


In [26]:
def set_capitals():
  # Getter that looks up the span text in the dictionary of country capitals
  get_capital = lambda span: CAPITALS.get(span.text)

  # Register the Span extension attribute "capital" with the getter get_capital
  Span.set_extension("capital", getter=get_capital, force=True)

  # Process the text and print the entity text, label and capital attributes
  doc = nlp("Czech Republic may help Slovakia protect its airspace")
  print([(ent.text, ent.label_, ent._.capital) for ent in doc.ents])
  

# NLM

In [29]:
FAST_START = True

In [79]:
from spacy.language import Language
#nlp = en_core_web_md.load()
def update_nlp():
  matcher = PhraseMatcher(nlp.vocab)
  CITIES = set(CITIES_IL + CITIES_API_city)

  # disable other pipes while doing the traing to speed it up - went down from 1:40 to 0:40 minutes
  if not FAST_START:
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):
      matcher.add("CITIES", None, *list(nlp.pipe(CITIES)))

  # country codes causes problems because many are similar to regular words (eg. 'IN' similar to 'in')
  #matcher.add("CITIES_API_country_code", None, *list(nlp.pipe(CITIES_API_country_code)))
  # due to duplicates with country codes ('IL' = Israel, and Ilinoie), we have to check manually for states
  #matcher.add("CITIES_API_state_code", None, *list(nlp.pipe(CITIES_API_state_code)))

  # Register the Span extension attributes "is_country" and "is_city" 
  check_country = lambda span: span.text.lower().translate(remove_punct_dict) in COUNTRIES
  Span.set_extension('is_country', getter=check_country, force=True)

  check_city = lambda span: span.text.lower().translate(remove_punct_dict) in CITIES
  Span.set_extension('is_city', getter=check_city, force=True)

  pipes = nlp.pipe_names
  if 'gpe_component' in pipes:
    nlp.remove_pipe('gpe_component')

  if spacy.__version__ > '3':
    @Language.component("gpe_component")
    def gpe_component(doc):
        # Create an entity Span with the label "GPE" for all matches
        matches = matcher(doc)
        try:
          doc.ents = [Span(doc, start, end, label="GPE") for match_id, start, end in matches]
        except ValueError as e: #[E103] Trying to set conflicting doc.ents: '(2, 4, 'GPE')' and '(3, 4, 'GPE')'. A token can only be part of one entity, so make sure the entities you're setting don't overlap.
          if 'E103' in e.args[0]:
            return doc  # use the standard model of Spacy, hopefully it works (as in the case "weather in new york") or  other rounds will get the proper result
          raise
        return doc

    # Add the new component to the pipeline
    nlp.add_pipe('gpe_component', first=True)
  else:
    def gpe_component(doc):
        # Create an entity Span with the label "GPE" for all matches
        matches = matcher(doc)
        try:
          doc.ents = [Span(doc, start, end, label="GPE") for match_id, start, end in matches]
        except ValueError as e: #[E103] Trying to set conflicting doc.ents: '(2, 4, 'GPE')' and '(3, 4, 'GPE')'. A token can only be part of one entity, so make sure the entities you're setting don't overlap.
          if 'E103' in e.args[0]:
            return doc  # use the standard model of Spacy, hopefully it works (as in the case "weather in new york") or  other rounds will get the proper result
          raise
        return doc

    # Add the new component to the pipeline
    nlp.add_pipe(gpe_component, first=True)
  #print(nlp.pipe_names)

update_nlp()


<IPython.core.display.Javascript object>

In [90]:
get_parts("weather in new york")
get_parts("weather in new york us")
get_parts("weather in las vegas")
get_parts("weather in las vegas, US", True)
get_parts("weather in las vegas, united states")
#get_parts("weather in kas vegas")
#get_parts("weather in kas vegas, US")

{'city': [('las vegas', 'honduras')]} True

weather, dep_=ROOT, ent_type=, head=weather, lemma_=weather, pos_=NOUN, tag_=NN
in, dep_=prep, ent_type=, head=weather, lemma_=in, pos_=ADP, tag_=IN
las, dep_=compound, ent_type=GPE, head=vegas, lemma_=las, pos_=PROPN, tag_=NNP
vegas, dep_=compound, ent_type=GPE, head=us, lemma_=vegas, pos_=PROPN, tag_=NNP
us, dep_=pobj, ent_type=GPE, head=in, lemma_=us, pos_=PROPN, tag_=NNP


<IPython.core.display.Javascript object>

las vegas, GPE='Countries, cities, states', is_city=True, is_country=False


<IPython.core.display.Javascript object>

us, GPE='Countries, cities, states', is_city=True, is_country=False
{'date': 'current', 'city': ('las vegas', 'united states'), 'country': 'united states'}


{'city': [('las vegas', 'united states')],
 'country': ['united states'],
 'date': 'current'}

# Get Parts (POS)

In [127]:
def check_gpe(entity, ents, ent_text, gpe):
        token = None
        for e in ents:
          if ent_text == e[0]:
            token = e[1] 
            break
        # verify the GPE is a true GPE for cases that a city has a name that has another meaninig
        if token and token.text != token.lemma_: # a true GPE cannot be lemmatized
          return 'continue', gpe, 0

        # for example there is a city in Denemark that has a name 'rain'
        if token and len(ents) > 1 and (token.head == token or token.pos_ not in ['PROPN', 'PRON', 'NOUN']) and not entity._.is_country:
          valid_gpe = False
          for e in ents:
            if ent_text != e[0]:
              tk = e[1]
              if tk.head == token:  # the token is connected to another gpe, so it is a valid gpe (for example, (paris, canada) are connected
                valid_gpe = True
                return 'break', gpe, 0
          if not valid_gpe:
            if DEBUG: p('A:', ent_text)
            return 'continue', gpe, 0

        ent_type = 'state'
        multiple_cities = False
        if DEBUG: p('0:', ent_text, entity._.is_city, entity._.is_country)
        if entity._.is_city and not entity._.is_country: # 'canada' is a country and also a city in Portugal
          ent_type = 'city'
          if ent_text in df_CITIES_API.index: 
            code_country = df_CITIES_API.loc[ent_text].country
            if type(code_country) != str:
              multiple_cities = True
              # see note why not to use code_country[0] which sometimes give 'Key Error'
              # https://stackoverflow.com/questions/46153647/keyerror-0-when-accessing-value-in-pandas-series            
              ent_country = countries_df.loc[code_country.iloc[0]].Name
            else:
              ent_country = countries_df.loc[code_country].Name
          elif ent_text in CITIES_IL:
            ent_country = 'israel'
          else:
            ent_country = ''
        elif entity._.is_country or ent_text in CITIES_API_country_code:
          ent_type = 'country'

        if ent_type not in gpe:
          gpe[ent_type] = []
        if ent_type == 'city':
           gpe[ent_type].append((entity.text, ent_country))
        else:
          gpe[ent_type].append(entity.text)
        return None, gpe, multiple_cities

In [130]:
def get_city_country(multiple_cities, gpe):
  if DEBUG: p('1:', multiple_cities, gpe)
  if multiple_cities or ('city' in gpe and len(gpe['city']) > 1):
    if 'country' in gpe:
      # verify country code of the city is correct, when there multiple cities
      for i, (city, city_country) in enumerate(gpe['city']):
        country = gpe['country'][0]
        if city_country != country:
          for code_country in df_CITIES_API.loc[city].country:
            if city_country == countries_df.loc[code_country].Name:
              # found a matching country
              gpe['city'][i] = (city, country)
              break
    else:
      check_capital = True
      if len(gpe['city']) == 2: # maybe enter a country which has city with that name (eg. Toronto, US = there is a city name 'us' in france and 'usa' in japan)
        # 'city': [('toronto', 'Canada'), ('us', 'france')]
        for i in range(1,-1,-1):
          city = gpe['city'][i][0].upper()
          if city in countries_df.index: #usually the 2nd "city" is the country
            city_country = countries_df.loc[city].Name
            j = 0 if i == 1 else 1
            gpe['city'] = [(gpe['city'][j][0], city_country)]
            gpe['country'] = city_country
            check_capital = False
            break
      if check_capital:
        # check if one of the cities is a capital or a large city, then use it
        #p(gpe, multiple_cities)
        for i, (city, city_country) in enumerate(gpe['city']):
          if city in capitals_df.index.values:
            # found a matching country
            city_country = capitals_df.loc[city].country
            gpe['city'][i] = (city, city_country)
            break
          if city in largest_df.index.values:
            # found a matching country
            city_country = largest_df.loc[city].country
            gpe['city'][i] = (city, city_country)
            break
  elif 'country' in gpe and 'city' in gpe:  # country with city, verify correct
      if DEBUG: p('2:', gpe)
      # verify country code of the city is correct, when user entered 
      city, city_country = gpe['city'][0]
      country = gpe['country'][0]
      if city_country != country:
        gpe['city'] = [(city, f'ERROR: city [{city}] not found in country [{country}] - try a different spelling of the city')]
  elif 'country' in gpe and 'city' not in gpe:  # country without city, get the capital
    country = gpe['country'][0]
    #p(country)
    if country in CAPITALS:
      gpe['city'] = [(CAPITALS[country], country)]
  return gpe

In [131]:
DEBUG = False
def get_parts(text, disp=False, verbose=True):
  global DEBUG
  doc = nlp(text.lower().translate(remove_punct_dict))
  date, gpe, ents = [], {}, []
  action = ''
  multiple_cities = False
  if disp: p()
  for token in doc:
    if disp:
      p(f'{token.text}, dep_={token.dep_}, ent_type={token.ent_type_}, head={token.head}, lemma_={token.lemma_}, pos_={token.pos_}, tag_={token.tag_}')
    if token.ent_type_ == 'GPE': ents.append((token.text.lower(), token))
  for entity in doc.ents:
      if disp:
        print(f"{entity.text}, {entity.label_}='{spacy.explain(entity.label_)}', is_city={entity._.is_city}, is_country={entity._.is_country}")
      ent_text = entity.text.lower().translate(remove_punct_dict)
      if entity.label_ == 'DATE': date.append(ent_text)
      if entity.label_ == 'GPE':
        act, gpe, multiple_cities = check_gpe(entity, ents, ent_text, gpe)
        if act == 'continue': continue
        if act == 'break': break
  if len(date) == 0:
    date = 'current'

  gpe = get_city_country(multiple_cities, gpe)

  result = {}
  result['date'] = date
  result.update(gpe)
  if disp:
    p(result)
  elif verbose and 'city' not in result:
    p(f'FAILED: org [{text}], {result}')
  return result

#DEBUG = False
if TEST:
  get_parts("Today weather in a Ra-anana")
  get_parts("Today weather in a Ra'anana")  
  get_parts("Today weather in a Raanana")
  get_parts("weather in toronto") # there is a city toronto in USA and toronto is not the capital of Canada so we take the first city in the db
  get_parts("weather in toronto, canada")
  get_parts("weather paris next 5 days")
  get_parts("tell me how is the weather in the ra- anana in the")
  get_parts("rain in toronto")
  get_parts("weather in toronto, us")
  get_parts("weather in toronto, canada")
  get_parts("weather in jenin")
  get_parts("weather in jenin, israel", True)
  get_parts('haifa, israel')
  DEBUG = False
  get_parts("Wheather forecast in Tel aviv")
get_parts("paris Canada")
get_parts("paris", True)


paris, dep_=compound, ent_type=GPE, head=canada, lemma_=paris, pos_=PROPN, tag_=NNP
canada, dep_=ROOT, ent_type=GPE, head=canada, lemma_=canada, pos_=PROPN, tag_=NNP


<IPython.core.display.Javascript object>

paris, GPE='Countries, cities, states', is_city=True, is_country=False


<IPython.core.display.Javascript object>

canada, GPE='Countries, cities, states', is_city=True, is_country=True
{'date': 'current', 'city': [('paris', 'ERROR: city [paris] not found in country [canada] - try a different spelling of the city')], 'country': ['canada']}

paris, dep_=ROOT, ent_type=GPE, head=paris, lemma_=paris, pos_=PROPN, tag_=NNP


<IPython.core.display.Javascript object>

paris, GPE='Countries, cities, states', is_city=True, is_country=False
{'date': 'current', 'city': [('paris', 'france')]}


{'city': [('paris', 'france')], 'date': 'current'}

In [114]:
if TEST:
  get_parts("paris Canada")
  get_parts("las vegas, united states")
  get_parts("las vegas, US")  
  get_parts("weather in las vegas, united states")
  get_parts("weather in las vegas, US")
  get_parts("weather in new york")    
  get_parts("weather in new york us")    

{'city': [('las vegas', 'united states')],
 'country': 'united states',
 'date': 'current'}

In [32]:
largest_df.loc['miami'].country

'United States'

In [33]:
'miami' in df_CITIES_API.index
df_CITIES_API.loc['miami'].country
countries_df.loc['US'].Name

'united states'

In [34]:
DEBUG = False
if TEST:
  get_parts("please tell me if it will rain tonight in Rain")
  get_parts("tomorrow and today rain in spain")
  get_parts("tomorrow rain in madrid")
  get_parts("tomorrow rain in beersheba")
  get_parts("tomorrow rain in israel")
  get_parts("weather paris Canada")
  get_parts("weather paris, Canada")
  get_parts("tomorrow rain in yafo")

In [35]:
if TEST:
  get_parts("Today weather in a Ra-anana")
  get_parts("weather in tel aviv")
  get_parts("tomorrow weather in haifa")
  get_parts("tomorrow rain in beersheba")
  get_parts("weather paris today and tomorrow")
  get_parts("weather paris")
  get_parts("weather paris in December")

In [36]:
get_parts("ra'anana")
get_parts("Today weather in a Tel Aviv")
get_parts("weather paris")
get_parts("paris weather ")
get_parts('Paris weather in')
get_parts("weather shelomi")

{'city': [('paris', 'france')]} True
{'city': [('paris', 'france')]} True
{'city': [('paris', 'france')]} True


{'city': [('shelomi', 'israel')], 'date': 'current'}

# Run the bot

In [37]:
nltk.download(['punkt', 'stopwords'])
stopwords = nltk.corpus.stopwords.words('english')

<IPython.core.display.Javascript object>

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


<IPython.core.display.Javascript object>

> when similarity was .075 it matched the sentence

>"tell me how is the weather in the ra- anana in the"

>to "what's up" of tag ["greeting"] so we increased it to 0.85

## Google search

In [38]:
import bs4
def get_google(text):
  url = "https://google.com/search?q=" + text + '&hl=en'
  request_result = requests.get( url )
  soup = bs4.BeautifulSoup(request_result.text, "html.parser")
  spans = soup.find_all('span', class_='BNeawe')
  new_text = ''
  for span in spans: 
    if isinstance(span.contents[0], bs4.NavigableString):
      new_text += str(span.contents[0]) + ' '
  return new_text
get_google("weather ra anana")

"Ra'anana, Israel  /  Weather "

In [117]:
GOOGLE = True
do = True
GREETING, NOANSWER = 0, 3
#p(intents['intents'][GREETING]['responses'][0])
min_similarity = 0.84
VERBOSE = 0

while do:
  user_msg = input().lower().strip()
  if user_msg == '':
    answers = intents['intents'][NOANSWER]['responses']
    #p("WeatherBot: " + random.choice(answers))
    continue
  round = 0
  answer = ''
  org_msg = user_msg
  tag = ''
  while round < 5 and do:
    if VERBOSE: p(f'round {round}: {user_msg}')
    doc1 = nlp(user_msg)
    answers, tag = None, None
    top_similarity = 0
    # verify not an erroneous finding (if we have GPE in the doc we assume it is not a general request)
    if len([ent for ent in doc1.ents if ent.label_ == 'GPE']) == 0:
      for ints in intents['intents']:
        #p(ints)
        for text in ints['patterns']:
          doc2 = nlp(text)
          # Get the similarity of doc1 and doc2
          if doc1.vector_norm == 0 or doc2.vector_norm == 0:
            continue
          similarity = doc1.similarity(doc2)
          if VERBOSE > 2: p(f"{similarity}, {doc2}, {[ints['tag']]}")
          if similarity > top_similarity:
            top_similarity = similarity
            answers = ints['responses']
            tag = ints['tag']
    #  p(top_similarity, tag)
    #  p(answers)
      if top_similarity > min_similarity:
        if tag == "goodbye":
          do = False
        if tag in ["goodbye", "greeting", "thanks", "options"]:
          answer = random.choice(answers)
          break
    if round == 2 and GOOGLE: round = 3 # to know we got the answer from get_parts()
    parts = get_parts(user_msg, VERBOSE > 1, False)
    if VERBOSE:  p('x:', parts)
    if 'date' in parts and 'city' in parts and 'ERROR' not in parts['city'][0][1]:
      #p(parts['city'][0][0], parts['city'])
      #p('calling get_weather')
      answer = get_weather(parts['date'], parts['city'], action=None)
    elif 'city' in parts and 'ERROR' in parts['city'][0][1]:
      answer = parts['city'][0][1]
    elif round == 0:
      round = 1
      user_msg = ''
      space = ' '
      for token in doc1:
        if token.text in stopwords: continue
        text = token.text.lower().translate(remove_punct_dict)
        if text == '': continue
        if text[-1] == '-': # take care of 'ra- anana'
          user_msg += space + text[:-1]
          space = ''
        else:
          user_msg += space + text
          space = ' '
      continue
    elif round == 1:
      round = 2
      user_msg = get_google(org_msg)
      if user_msg != '':
        continue
    if round == 2 or round == 3:  # need this round because in round 2 get_google('ביי') returns empty string
      round = 4
      user_msg = translator.translate(org_msg) # deafaul translation is to english - can set source/dest langauge: src="de"/ dest="he"
      if VERBOSE:  p(f"{user_msg.origin} ({user_msg.src}) --> {user_msg.text} ({user_msg.dest})")
      user_msg = user_msg.text
      if user_msg != '':
        continue
    if round >= 3 and not answer == '' and not 'ERROR' in answer:
      answer += ' (from Google)'
    break
  if answer == '': answer = "You need to tell me a city to check (maybe spell it differently)"  
  p(f"WeatherBot: {answer}")
  #p(tag)
# weather in jenin, israel  
# 'קשאיקר ןמ ןדרשקך (weather in israel un hebrew)

weather in las vegas, US
WeatherBot: In Las vegas, United states the current weather is: clear sky
bye
WeatherBot: Bye! Come back again soon.


In [40]:
if TEST:
  # example of successful chat although get parts failed
  get_parts("weather in tel-aviv")
  get_parts("weather in TelAviv")   # gave answer for Israel/Jerusalem
  get_parts("what is tel aviv")
  get_parts("wether in tel aviv")
  get_parts("please tell me if it will rain tonight in Rain")

In [41]:
if TEST:
  # fixed in round1 of the bot (remove stop words and dashes followed by a space '- ')
  get_parts("weather ra- anana") # for google
  get_parts("tell me how is the weather in the ra- anana in the")

# TODO:

1. DONE: display country when getting result of weather so the user knows it is the right country - if there multiple cities

2. DONE: load jsons from github - (one file too large so download it from a shared goole link on my google drive)

3. DONE: Use flask

4. DONE: if fails to understand try round 2 - removing stop words (punctuation akready done by spacy), then round 3 use google to try to fix typo mistake and only if all fails display a message to user

5. To identify if it is a question or not - must be question

Easy:
1. Wheather forecast in Tel aviv - DONE partially, what do you mean by 'forecast'

1. How hot is it in Herzliya

1. Today’s temperature in Netanya

1. Moist in Hadera

1. How may degrees tonight in Hadera 	Ofakim

1. Weekly forecast for Ramat Hashron

1. Israel desert forecast

1. Weather in Ramat gan

1.  Moist in Givataim

1. Wave height in Ashdod

1. Moist ashkelon

1. Night degrees in Eilat

10 שאלות מורכבות על מזג אויר

1. Newt 10 days forecast in Herzliya and Raanana

1. Is there a weather forecast on Herzliya

1. Would like to get weather forecast on Hadera next week

1. North Israel weather

1. Monthly average forecast for north israel

1. Monthly average forecast on Thailand

10 שאלות שגויות לבדיקה מה ניתן

1. What are the optional cities for weather forecast

1. Who is the first prime minister of Israel

1. Who where the 3 stooges

1. What is a banana

1. Who was Napoleon

1. When was photography invented

1. Are there aliens

Extras:
1. בדיקה על תוצרי שליפה במקרה של עיר שקיימת ביותר ממדינה אחת

1. 	במשלוח של עיר כמו PARIS, התקבלו תוצאות הן על צרפת והן ארצות הברית.

1. איזה פונקציונאליות נוספת ה API יודע לתת - כרגע נבחר API אחד שיודע לעבוד עם עיר בלבד.

1. להבין את שיטות העבודה של ה API (מה קורה במידה ומבקשים תחזית “מחר” - מה הוא יודע להחזיר) - לא רלבנטי 

1. עובדים עם API בסיסי

# QUESTION:

2. Are we to handle 'Ra-anana'? we handle 'Raanana'
3. DONE - There is a city toronto in USA and toronto is not the capital of Canada so we take the first city in the db (we check now if it is one of the 100 largest cities in the world)

> spacy.__version__ 
2.2.4

> get_parts('haifa, israel', True)

haifa, dep_=compound, ent_type=GPE, head=israel, lemma_=haifa, pos_=PROPN, tag_=NNP

israel, dep_=ROOT, ent_type=GPE, head=israel, lemma_=israel, pos_=PROPN, tag_=NNP

haifa, GPE='Countries, cities, states', is_city=True, is_country=False

israel, GPE='Countries, cities, states', is_city=False, is_country=True

{'date': 'current', 'city': [('haifa', 'israel')], 'country': ['israel']}

{'city': [('haifa', 'israel')], 'country': ['israel'], 'date': 'current'}

# EXAMPLES

0 You:	weather in jenin, israel

1	Bot:	ERROR: city [jenin] not found in country [israel] - try a different spelling of the city

2	You:	weather in hebron, israel

3	Bot:	In Hebron, Israel the current weather is: clear sky

4	You:	weather in jenin

5	Bot:	In Jenin, Poland the current weather is: overcast clouds

6	You:	weather in hebron

7	Bot:	In Hebron, Palestine, state of the current weather is: clear sky

8 weather isral

9 WeatherBot: In Jerusalem, Israel the current weather is: clear sky (from Google)

10 中國的天氣

In Beijing, China the current weather is: overcast clouds (from Google)

11 погода в китае
12 погода в россии

# TEST

In [42]:
user_msg = 'test'
answer = 'answer'
session = {'df_chat':[]}
session['df_chat'] = [[2, 'You: ', user_msg], [1, 'Bot: ', answer]] + session['df_chat']
session['index'] = 2 
df_chat = pd.DataFrame(session['df_chat'], columns=['index', 'ID', 'Conversation'])
df_chat.set_index('index', inplace=True)
df_chat


<IPython.core.display.Javascript object>

,ID,Conversation
index,,
2,You:,test
1,Bot:,answer


In [43]:
index = session['index']
session['index'] = index + 2
session['df_chat'] = [[index + 2, 'You: ', user_msg], [index + 1, 'Bot: ', answer]] + session['df_chat']
df_chat = pd.DataFrame(session['df_chat'], columns=['index', 'ID', 'Conversation'])
df_chat.set_index('index', inplace=True)
df_chat

<IPython.core.display.Javascript object>

,ID,Conversation
index,,
4,You:,test
3,Bot:,answer
2,You:,test
1,Bot:,answer


In [44]:
df_chat.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>ID</th>\n      <th>Conversation</th>\n    </tr>\n    <tr>\n      <th>index</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>4</th>\n      <td>You:</td>\n      <td>test</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Bot:</td>\n      <td>answer</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>You:</td>\n      <td>test</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Bot:</td>\n      <td>answer</td>\n    </tr>\n  </tbody>\n</table>'

In [45]:
st = df_chat.style
